# Webexport
!! tippecanoe installieren !!

```
brew install tippecanoe
```

In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import json
import consts
from utils import categorize_time_to_half_hour, categorize_time_to_half_hour_numerical, dt_to_time, time_to_time

In [2]:
# Load timedata
time_data = json.load(open(consts.PATH_TIMEDATA_HEX, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(consts.PATH_HEXAGON)

# Reduce precision
#gdf_grid_raw['geometry'] = gdf_grid_raw['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

In [3]:
# Serialize Timedata
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'name': x['name'],
    'sunrise_current_dt': x['sunrise']['current']['latest'],
    'sunrise_summertime': x['sunrise']['summer']['latest'],
    'sunrise_wintertime': x['sunrise']['winter']['latest']
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert strings to time
df_timedata['sunrise_current'] = df_timedata['sunrise_current_dt'].apply(dt_to_time)
df_timedata['sunrise_summertime'] = df_timedata['sunrise_summertime'].apply(time_to_time)
df_timedata['sunrise_wintertime'] = df_timedata['sunrise_wintertime'].apply(time_to_time)

# Current System
df_timedata['sunrise_current_category'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_current_numerical'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Summertime
df_timedata['sunrise_summertime_category'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_summertime_numerical'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Wintertime
df_timedata['sunrise_wintertime_category'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_wintertime_numerical'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Merge Dataframes
gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')

## Export GeoJSON

In [4]:
gdf_grid.to_file(consts.PATH_GEOJSON_EXPORT)